In [124]:
import sklearn as sk
import pandas as pd
import numpy as np

In [125]:
DATAFILE = "./dataset/weather_data.csv"
TESTDATASET = "./dataset/unknown_data.csv"
dataset = pd.read_csv(DATAFILE)
testDataset =pd.read_csv(TESTDATASET)

In [126]:
dataset.head()

,row ID,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Row0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0
1,Row1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0
2,Row2,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0
3,Row3,Albury,14.6,29.7,0.2,NaN,NaN,WNW,56.0,W,...,55.0,23.0,1009.2,1005.4,NaN,NaN,20.6,28.9,No,0
4,Row4,Albury,7.7,26.7,0.0,NaN,NaN,W,35.0,SSE,...,48.0,19.0,1013.4,1010.1,NaN,NaN,16.3,25.5,No,0


In [127]:
dataset.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow
count,99073.000000,99286.000000,98537.000000,56985.00000,52199.000000,93036.000000,98581.000000,97681.000000,98283.000000,97010.000000,89768.000000,89780.000000,61944.000000,59514.000000,98902.000000,97612.000000,99516.000000
mean,12.176266,23.218513,2.353024,5.46132,7.615090,39.976966,14.004849,18.650464,68.866376,51.433296,1017.684638,1015.286204,4.447985,4.519122,16.970041,21.681340,0.224677
std,6.390882,7.115072,8.487866,4.16249,3.783008,13.581524,8.902323,8.801827,19.074951,20.777616,7.110166,7.045189,2.886580,2.716618,6.488961,6.931681,0.417372
min,-8.500000,-4.100000,0.000000,0.00000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.500000,978.200000,0.000000,0.000000,-7.000000,-5.100000,0.000000
25%,7.600000,17.900000,0.000000,2.60000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1013.000000,1010.500000,1.000000,2.000000,12.300000,16.600000,0.000000
50%,12.000000,22.600000,0.000000,4.80000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.700000,1015.300000,5.000000,5.000000,16.700000,21.100000,0.000000
75%,16.800000,28.200000,0.800000,7.40000,10.600000,48.000000,19.000000,24.000000,83.000000,65.000000,1022.400000,1020.000000,7.000000,7.000000,21.500000,26.400000,0.000000
max,33.900000,48.100000,371.000000,86.20000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.000000,1039.600000,9.000000,9.000000,40.200000,46.700000,1.000000


In [128]:
dataset.count().sort_values()

Sunshine         52199
Evaporation      56985
Cloud3pm         59514
Cloud9am         61944
Pressure9am      89768
Pressure3pm      89780
WindDir9am       92510
WindGustDir      92995
WindGustSpeed    93036
WindDir3pm       96868
Humidity3pm      97010
Temp3pm          97612
WindSpeed3pm     97681
Humidity9am      98283
Rainfall         98537
RainToday        98537
WindSpeed9am     98581
Temp9am          98902
MinTemp          99073
MaxTemp          99286
row ID           99516
Location         99516
RainTomorrow     99516
dtype: int64

In [129]:
dataset.shape

(99516, 23)

## First, the RainToday attribute is an binary value where it only store Yes/No for answer. So I will manually binarise the RainToday attribute using replace() function in pd

## Drop these columns (Sunshine, Evaporation, Cloud3pm/9am) as they have to much NA value

In [130]:
dataset = dataset.drop(columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am','row ID'],axis=1) 
testDataset = testDataset.drop(columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am','row ID'],axis=1) 
print(dataset.shape)
print(testDataset.shape)

(99516, 18)
(42677, 17)


# Impute the data for the testDataset

In [131]:
dataset = dataset.dropna()
testDataset.shape

(42677, 17)

In [132]:
from scipy import stats
z = np.abs(stats.zscore(dataset._get_numeric_data()))
print(z)
dataset= dataset[(z < 3).all(axis=1)]
print(dataset.shape)

        MinTemp   MaxTemp  Rainfall  WindGustSpeed  WindSpeed9am  \
0      0.119802  0.105934  0.207977       0.240913      0.576967   
1      0.842097  0.209274  0.278039       0.240913      1.339686   
2      0.777100  1.240864  0.161269       0.015814      0.980314   
3      0.312182  0.868346  0.254685       1.141306      0.457176   
4      0.794002  0.438516  0.278039       0.434382      1.100105   
...         ...       ...       ...            ...           ...   
99511  0.745907  0.421143  0.278039       0.015814      0.457176   
99512  1.467331  0.263539  0.278039       0.734513      0.021987   
99513  1.579553  0.034296  0.278039       0.734513      0.261569   
99514  1.451299  0.237929  0.278039       1.409809      0.261569   
99515  1.162730  0.467171  0.278039       0.284317      0.740732   

       WindSpeed3pm  Humidity9am  Humidity3pm  Pressure9am  Pressure3pm  \
0          0.523188     0.188186     1.381221     1.385198     1.143726   
1          0.290050     1.237542 

In [133]:
dataset.isna().sum()
testDataset.isna().sum()

Location            0
MinTemp           194
MaxTemp            92
Rainfall          427
WindGustDir      2809
WindGustSpeed    2790
WindDir9am       3007
WindDir3pm       1130
WindSpeed9am      413
WindSpeed3pm      795
Humidity9am       541
Humidity3pm      1104
Pressure9am      4266
Pressure3pm      4245
Temp9am           290
Temp3pm           822
RainToday         427
dtype: int64

In [134]:
dataset.shape

(75601, 18)

# Performing standardization the data

In [135]:
from sklearn.preprocessing import StandardScaler

# create a scaler object
std_scaler = StandardScaler()


def returnListOfNumericCols(data):
    numericColumns = []

    for col in data.columns:
        if(data[col].dtype == 'float64'): # if the type of the col is numeric
            numericColumns.append(col)

    return numericColumns

In [136]:
# Importing the SimpleImputer class
from sklearn.impute import SimpleImputer
 
imputer = SimpleImputer(missing_values = np.nan,strategy ='mean')
 
 
print(testDataset)
for col in returnListOfNumericCols(dataset):
        testDataset[col] = imputer.fit_transform(testDataset[col].values.reshape(-1, 1))
 
# Imputing the data    
 
print("Imputed Data : \n", testDataset)

      Location  MinTemp  MaxTemp  Rainfall WindGustDir  WindGustSpeed  \
0       Albury     12.9     25.7       0.0         WSW           46.0   
1       Albury      9.2     28.0       0.0          NE           24.0   
2       Albury     14.3     25.0       0.0           W           50.0   
3       Albury      9.7     31.9       0.0         NNW           80.0   
4       Albury     15.9     18.6      15.6           W           61.0   
...        ...      ...      ...       ...         ...            ...   
42672    Uluru      2.4     19.1       0.0           E           33.0   
42673    Uluru      2.3     21.4       0.0          SE           22.0   
42674    Uluru      2.6     22.5       0.0           S           19.0   
42675    Uluru      7.4     20.6       0.0           E           35.0   
42676    Uluru      7.8     27.0       0.0          SE           28.0   

      WindDir9am WindDir3pm  WindSpeed9am  WindSpeed3pm  Humidity9am  \
0              W        WSW          19.0          

In [137]:


for col in returnListOfNumericCols(dataset):
    scalerData = std_scaler.fit_transform(dataset[col].values.reshape(-1, 1))
    scalerTestData = std_scaler.fit_transform(testDataset[col].values.reshape(-1, 1))
    
    dataset[col] =  pd.DataFrame(scalerData, columns=[col])
    testDataset[col] = pd.DataFrame(scalerTestData, columns=[col])
print(dataset.shape)
print(testDataset.shape)

(75601, 18)
(42677, 17)


In [138]:
dataset.head()

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Albury,0.135368,-0.106682,-0.228025,W,0.352712,W,WNW,0.692096,0.615139,0.192726,-1.389806,-1.495979,-1.233579,-0.075956,-0.048832,No,0
1,Albury,-0.826882,0.211509,-0.381104,WNW,0.352712,NNW,WSW,-1.395351,0.365562,-1.263322,-1.242612,-1.065598,-1.129124,-0.028470,0.321010,No,0
2,Albury,0.792906,1.252860,-0.125972,W,0.102254,ENE,NW,-1.003955,0.115986,0.785931,-0.850097,-1.035916,-1.397723,0.066502,1.119867,No,0
3,Albury,0.327818,0.876817,-0.330077,WNW,1.354545,W,W,0.561631,0.615139,-0.670117,-1.340741,-1.273368,-1.487256,0.509705,1.001518,No,0
4,Albury,-0.778770,0.442920,-0.381104,W,-0.398663,SSE,W,-1.134420,-0.258379,-1.047611,-1.536999,-0.650058,-0.785915,-0.170928,0.498534,No,0


# Binarise the categories data

In [139]:
dataset["RainToday"].replace({"No":0,"Yes":1},inplace = True)
testDataset["RainToday"].replace({"No":0,"Yes":1},inplace = True)



In [140]:
dataset.head()

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Albury,0.135368,-0.106682,-0.228025,W,0.352712,W,WNW,0.692096,0.615139,0.192726,-1.389806,-1.495979,-1.233579,-0.075956,-0.048832,0,0
1,Albury,-0.826882,0.211509,-0.381104,WNW,0.352712,NNW,WSW,-1.395351,0.365562,-1.263322,-1.242612,-1.065598,-1.129124,-0.028470,0.321010,0,0
2,Albury,0.792906,1.252860,-0.125972,W,0.102254,ENE,NW,-1.003955,0.115986,0.785931,-0.850097,-1.035916,-1.397723,0.066502,1.119867,0,0
3,Albury,0.327818,0.876817,-0.330077,WNW,1.354545,W,W,0.561631,0.615139,-0.670117,-1.340741,-1.273368,-1.487256,0.509705,1.001518,0,0
4,Albury,-0.778770,0.442920,-0.381104,W,-0.398663,SSE,W,-1.134420,-0.258379,-1.047611,-1.536999,-0.650058,-0.785915,-0.170928,0.498534,0,0


In [141]:
testDataset.head()

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday
0,Albury,0.107515,0.344855,-0.279918,WSW,0.456048,W,WSW,0.566827,0.847335,-1.631312,-1.049812,-1.483130,-0.975153,0.612963,0.217752,0.0
1,Albury,-0.469042,0.668078,-0.279918,NE,-1.216512,SE,E,-0.339247,-1.101224,-1.260453,-1.730319,0.002735,-0.359374,0.165402,0.697119,0.0
2,Albury,0.325672,0.246483,-0.279918,W,0.760150,SW,W,0.680086,0.618092,-1.048533,-1.584496,-1.185957,-1.050248,0.165402,0.421120,0.0
3,Albury,-0.391129,1.216152,-0.279918,NNW,3.040913,SE,NW,-0.792284,1.076577,-1.419392,-2.070572,-1.289967,-1.741122,0.196268,1.234591,0.0
4,Albury,0.574993,-0.652920,1.583922,W,1.596429,NNW,NNW,1.586159,1.076577,0.381922,2.012469,-3.459330,-3.333137,0.057370,-0.857193,1.0


In [142]:
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(dtype=int,sparse=False)
catCols = []
for col in dataset.columns :
    if(dataset[col].dtypes == 'object'):
        catCols.append(col)
print(catCols)

['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']


In [143]:
dataset = pd.get_dummies(dataset,columns=catCols)
testDataset = pd.get_dummies(testDataset,columns=catCols)



In [144]:
dataset.shape

(75601, 106)

In [145]:
datasetCols = []
testDatasetCols = []
for col in dataset.columns:
    datasetCols.append(col)
    

for col in testDataset.columns:
    testDatasetCols.append(col)

unknownColumns = set(datasetCols)
for col in testDatasetCols:
    if(col not in unknownColumns):
        testDataset = testDataset.drop(columns=[col],axis=1)


In [146]:
testDataset

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,0.107515,0.344855,-0.279918,0.456048,0.566827,0.847335,-1.631312,-1.049812,-1.483130,-0.975153,...,0,0,0,0,0,0,0,0,0,1
1,-0.469042,0.668078,-0.279918,-1.216512,-0.339247,-1.101224,-1.260453,-1.730319,0.002735,-0.359374,...,0,0,0,0,0,0,0,0,0,0
2,0.325672,0.246483,-0.279918,0.760150,0.680086,0.618092,-1.048533,-1.584496,-1.185957,-1.050248,...,0,0,0,0,0,0,0,1,0,0
3,-0.391129,1.216152,-0.279918,3.040913,-0.792284,1.076577,-1.419392,-2.070572,-1.289967,-1.741122,...,0,1,0,0,0,0,0,0,0,0
4,0.574993,-0.652920,1.583922,1.596429,1.586159,1.076577,0.381922,2.012469,-3.459330,-3.333137,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42672,-1.528659,-0.582654,-0.279918,-0.532283,0.340308,-0.184255,-0.518735,-1.341457,1.845208,1.653174,...,0,0,0,0,0,0,0,0,0,0
42673,-1.544242,-0.259431,-0.279918,-1.368563,-0.339247,-1.330466,-0.465755,-1.147027,1.384590,1.142527,...,0,0,0,0,1,0,0,0,0,0
42674,-1.497494,-0.104846,-0.279918,-1.596639,-0.565765,-1.330466,-0.518735,-1.341457,1.102275,0.932261,...,0,0,0,0,0,0,0,0,0,0
42675,-0.749529,-0.371857,-0.279918,-0.380232,0.113790,-0.184255,-0.306816,-0.903989,1.429166,1.217622,...,0,0,0,0,0,0,0,0,0,0


In [147]:
X = dataset.loc[:,dataset.columns!='RainTomorrow']
y = dataset.RainTomorrow
X = np.nan_to_num(X)

In [148]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [149]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=5,max_leaf_nodes=3) # max_depth=5,max_leaf_nodes=3
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, max_leaf_nodes=3)

In [150]:
y_pred = clf.predict(X_test)

In [151]:
clf.score(X_test,y_test)

0.7907518120734353

In [152]:
from sklearn.neighbors import KNeighborsClassifier

Ksclf = KNeighborsClassifier(n_neighbors = 5)
Ksclf.fit(X_train, y_train)

Ksclf.classes_


array([0, 1])

In [153]:
from sklearn.ensemble import RandomForestClassifier

# Construct a random forest classifier.
rdFrClf = RandomForestClassifier(n_estimators = 50, max_features="auto", oob_score=True)
rdFrClf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50, oob_score=True)

In [154]:
rdFrClf.score(X_test,y_test)

0.7977884768001693

In [155]:
from sklearn.linear_model import LogisticRegression


# Construct a random forest classifier.
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

/home/huukhang1512/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [158]:
from sklearn.svm import NuSVC

SvmClf = NuSVC(kernel='rbf',nu=0.01)
SvmClf.fit(X_train, y_train)

In [ ]:
testDataset.isnull().sum().sort_values(ascending = False)

RainToday                    427
MinTemp                        0
WindGustDir_SE                 0
WindDir9am_NE                  0
WindDir9am_N                   0
                            ... 
Location_Mildura               0
Location_MelbourneAirport      0
Location_Melbourne             0
Location_Launceston            0
WindDir3pm_WSW                 0
Length: 105, dtype: int64

In [ ]:
testDataset['RainToday'] = testDataset['RainToday'].fillna(0)

In [ ]:
log_reg.score(X_test,y_test)

0.793450082006243

In [ ]:
testDataset.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,0.107515,0.344855,-0.279918,0.456048,0.566827,0.847335,-1.631312,-1.049812,-1.483130,-0.975153,...,0,0,0,0,0,0,0,0,0,1
1,-0.469042,0.668078,-0.279918,-1.216512,-0.339247,-1.101224,-1.260453,-1.730319,0.002735,-0.359374,...,0,0,0,0,0,0,0,0,0,0
2,0.325672,0.246483,-0.279918,0.760150,0.680086,0.618092,-1.048533,-1.584496,-1.185957,-1.050248,...,0,0,0,0,0,0,0,1,0,0
3,-0.391129,1.216152,-0.279918,3.040913,-0.792284,1.076577,-1.419392,-2.070572,-1.289967,-1.741122,...,0,1,0,0,0,0,0,0,0,0
4,0.574993,-0.652920,1.583922,1.596429,1.586159,1.076577,0.381922,2.012469,-3.459330,-3.333137,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
prediction = log_reg.predict(testDataset)
# Rownp.arange(len(predicted_values))
predicted_values=pd.DataFrame(prediction,columns=["Predict-RainTomorrow"])
# predicted_values["row ID"] = predicted_values.index

In [ ]:
testDataset =pd.read_csv(TESTDATASET)
predicted_values["row ID"] = testDataset["row ID"]

In [ ]:
predicted_values = predicted_values.set_index("row ID")

In [ ]:
predicted_values

,Predict-RainTomorrow
row ID,
Row0,0
Row1,0
Row2,0
Row3,0
Row4,1
...,...
Row43633,0
Row43634,0
Row43635,0


In [ ]:
predicted_values.to_csv("./13717335.csv",index=True)